In [1]:
import os
from delta.tables import *
from delta import *
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession

builder = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8eba86fe-76ba-4da0-8243-059e954b0321;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 163ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-8eba86fe-76ba-4da0-8243-059e954b0321
	confs: [default]
	0 artifacts copied, 3 already retrieved (

Atividade 4: Uso dos dados

In [5]:
#Abertura das tabelas geradas pela ETL produzida
df = spark.read.parquet("ml/gold")

#Visualizada na vertical devido a limitações quanto ao tamanho do monitor
df.orderBy("id").show(4, vertical = True)
display(df)

-RECORD 0-----------------------------
 id            | 1                    
 nome_completo | Aleksandr_Crighton   
 email         | acrighton0@instag... 
 valor_pedido  | 71.31                
 data_pedido   | 2021-08-21 14:18:35  
 tipo_cc       | jcb                  
 regiao        | Saitama Prefecture   
 cidade        | Yashio               
 pais          | Japan                
-RECORD 1-----------------------------
 id            | 2                    
 nome_completo | Tad_Arangy           
 email         | tarangy1@studiopr... 
 valor_pedido  | 28.12                
 data_pedido   | 2021-10-28 22:44:27  
 tipo_cc       | maestro              
 regiao        | Santa Fe             
 cidade        | Rosario              
 pais          | Argentina            
-RECORD 2-----------------------------
 id            | 3                    
 nome_completo | Wake_Samart          
 email         | wsamart2@ning.com    
 valor_pedido  | 148.18               
 data_pedido   | 2022-06-

DataFrame[id: int, nome_completo: string, email: string, valor_pedido: double, data_pedido: timestamp, tipo_cc: string, regiao: string, cidade: string, pais: string]

In [34]:
#Agrupamento dos dados referentes a bandeira dos cartões utilizados e contagem das bandeiras visa e mastercard

mastercard = df.filter(df.tipo_cc == 'mastercard').count()
visa = df.filter(df.tipo_cc == 'visa').count()
df.groupBy('tipo_cc').count().withColumnRenamed("tipo_cc","bandeira_cartao").withColumnRenamed("count","compras").show()

print(f"Total de compras com mastercard: {mastercard}")
print(f"Total de compras com visa: {visa}")

+--------------------+-------+
|     bandeira_cartao|compras|
+--------------------+-------+
|                 jcb|    405|
|       visa-electron|     51|
|             maestro|     77|
|                solo|     17|
|     americanexpress|     56|
| diners-club-enroute|     47|
|      china-unionpay|     48|
|            bankcard|     38|
|              switch|     61|
|          mastercard|     86|
|        instapayment|     16|
|diners-club-carte...|     32|
|                visa|     27|
|               laser|     26|
|   diners-club-us-ca|      6|
|diners-club-inter...|      7|
+--------------------+-------+

Total de compras com mastercard: 86
Total de compras com visa: 27


In [29]:
#Agrupamento das vendas de acordo com o dia de compra
df.groupBy(to_date('data_pedido').alias("data_pedido")).count().withColumnRenamed("count","qtd_pedidos").sort(to_date('data_pedido'), ascending=True).show()

+-----------+-----------+
|data_pedido|qtd_pedidos|
+-----------+-----------+
| 2021-07-27|          4|
| 2021-07-28|          5|
| 2021-07-29|          2|
| 2021-07-30|          2|
| 2021-07-31|          3|
| 2021-08-01|          5|
| 2021-08-02|          2|
| 2021-08-03|          4|
| 2021-08-04|          1|
| 2021-08-05|          1|
| 2021-08-06|          1|
| 2021-08-07|          4|
| 2021-08-08|          3|
| 2021-08-09|          2|
| 2021-08-10|          2|
| 2021-08-11|          2|
| 2021-08-12|          2|
| 2021-08-14|          3|
| 2021-08-16|          2|
| 2021-08-17|          3|
+-----------+-----------+
only showing top 20 rows

